In [0]:
%pip install openai unidecode gspread==5.12.4 tiktoken mlflow
%restart_python
%load_ext autoreload
%autoreload 2 

In [0]:
%restart_python
%load_ext autoreload
%autoreload 2 

In [0]:
from abc import ABC, abstractmethod
import time
import mlflow
from contextlib import contextmanager
import json
import pandas as pd
import datetime
import re
import openai
from openai import OpenAI
import gspread
import random
import logging
import os
from unidecode import unidecode
from pathlib import Path
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *
from functools import reduce
from typing import *
import tiktoken
import io


from general_config import *
from ml_tracker import MLTracker
from aso_config import *
from aso_localizer import *
#from in_game_localizer import InGameLocalizer
#from InGame_Config import * 

In [0]:
%run "./authenticationScript"

In [0]:
#TODO update authenticatinon script and secrets!!
gsheet_client = get_gspread_client_from_secret_old()
gpt_client = get_model_client() 

In [0]:
#DONE
#{'Game':"Panda Pop",'URL':"https://docs.google.com/spreadsheets/d/1WYl_MiwawlGcgRrXxLNN9WVYgWqqwu393n-9uO7aPhY/edit?gid=980819314#gid=980819314"},{'Game':"Disney Magic Match",'URL':"https://docs.google.com/spreadsheets/d/1r8jIynEvuHz9rz_Gu90dvOj7AcTCSz6A727gTP9DRbY/edit?gid=980819314#gid=980819314"},{'Game':"Cookie Jam",'URL':"https://docs.google.com/spreadsheets/d/1zXG3nuxrfHtKRw98COt1kwjecAIjQ4iGl0b3NDMes1w/edit?gid=980819314#gid=980819314"},
#{'Game':"Harry Potter: Hogwarts Mystery",'URL':"https://docs.google.com/spreadsheets/d/1PRIm4_HOVjmLz3Sm6gIpRO8jJDEc60eG21WcX7t9jwg/edit?gid=980819314#gid=980819314"}

In [0]:
##{'Game':"Disney Emoji Blitz",'URL':"https://docs.google.com/spreadsheets/d/1rRS6jnCX16tpWpXxknvGmui6KAwLO8-l7Qxsj6j6f98/edit?gid=980819314#gid=980819314"

In [0]:
#october_urls = [{'Game':"Disney Emoji Blitz",'URL':"https://docs.google.com/spreadsheets/d/1rRS6jnCX16tpWpXxknvGmui6KAwLO8-l7Qxsj6j6f98/edit?gid=980819314#gid=980819314"},]

In [0]:
october_urls = [{'Game':'Cookie Jam', "URL": "https://docs.google.com/spreadsheets/d/1zXG3nuxrfHtKRw98COt1kwjecAIjQ4iGl0b3NDMes1w/edit?usp=sharing"}]

In [0]:
base_request = {"RowFingerprint":"aso_october_cj",
                    "Timestamp":datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    "SubmitterEmail":"krista@jamcity.com",
                    "DueDate":"9/4/25",
                    'LocType':"ASO", 
                    'TargetLanguages':"",
                    "QAFlag":""}

cfg = {
    "input": 
        {
            "required_tabs": ["ios","android"],
            "ios_header_rows": 3, 
            "android_header_rows": 3
        },
    ##add more formatting data for header rows
    "char_limit_policy": "strict",
    "output_sheets":
        ["formatted ios", "formatted android", "long results",'wide results'],
    ##add more formatting info for output sheets
    "log_full_artifacts":True}

localizers= []
for i in october_urls:
    i = {**base_request, **i}
    localizer = ASOLocalizer(request = i, gsheet_client =gsheet_client, gpt_client = gpt_client, cfg=cfg)
    localizers.append(localizer)

In [0]:
localizers[0].run()

In [0]:
#mlflow.end_run("3739efa5a69945e09c0e817f98137087")
run = mlflow.active_run()
mlflow.end_run()

In [0]:
print(mlflow.active_run())
mlflow.end_run()

In [0]:
results

In [0]:
results = []
for loc in localizers:
    print(f"Running for {loc.request['Game']}")
    results.append(loc.run())
    #postprocessed = loc.postprocess(results)
    #print(loc.write_outputs(postprocessed))

In [0]:
results

In [0]:
"""
request =  {"RowFingerprint":"testing_aso_run_v2",
                    "Timestamp":datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    "SubmitterEmail":"krista@jamcity.com",
                    "DueDate":"8/29/25",
                    'LocType':"ASO", 
                    'Game':"Panda Pop", 
                    'TargetLanguages':"",
                    "URL":"https://docs.google.com/spreadsheets/d/1QsZqlJ_vZC5fYIKnSFdhkUxi-UAIRNzTB24DDyoZMhU/edit?gid=980819314#gid=980819314",
                    "QAFlag":""}


cfg = {
    "input": 
        {
            "required_tabs": ["ios","android"],
            "ios_header_rows": 3, 
            "android_header_rows": 3
        },
    ##add more formatting data for header rows
    "char_limit_policy": "strict",
    "output_sheets":
        ["formatted ios", "formatted android", "long results",'wide results'],
    ##add more formatting info for output sheets
    }"""

In [0]:
localizer = ASOLocalizer(request = request, gsheet_client =gsheet_client, gpt_client = gpt_client, cfg=cfg)

In [0]:
#localizer.run()